In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import metrics
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
#read the training data
train = pd.read_csv(
    open(
        r'C:\Users\cmgas\OneDrive\git\Best-Buy-Case\train_Croston.csv', 'rb'),
)

In [4]:
#convert SALES_DATE to datetime module
train['SALES_DATE'] = pd.to_datetime(train['SALES_DATE'])

In [5]:
#impute missing rows for the training set by adding a row for every missing date between each SKU's minimum and maximum date
missing_dates_1 = []
missing_skus_1 = []

for i in list(train['Encoded_SKU_ID'].unique()):
    #filtered df for each SKU
    df_filt = train[train['Encoded_SKU_ID'] == i]

    #take the min and max of the SKU dates 
    start_date = df_filt['SALES_DATE'].min()
    end_date = df_filt['SALES_DATE'].max()
    dates_present = df_filt['SALES_DATE'].unique()

    n = (end_date - start_date).days
    dream_dates = [end_date - dt.timedelta(days=x) for x in range(n)]

    for date_val in dream_dates:
        if date_val in dates_present:
            continue
        else:
            missing_dates_1.append(date_val)
            missing_skus_1.append(i)

In [6]:
#zip together missing values and impute Daily_Units as 0
missing_values_1 = pd.DataFrame(zip(missing_dates_1, missing_skus_1), columns = ['SALES_DATE', 'Encoded_SKU_ID'])
missing_values_1['DAILY_UNITS'] = 0

In [7]:
#drop all exogenous features from the dataset
train.drop('SUBCLASS_NAME', inplace=True, axis=1)
train.drop('CLASS_NAME', inplace=True, axis=1)
train.drop('ML_NAME', inplace=True, axis=1)
train.drop('CATEGORY_NAME', inplace=True, axis=1)
train.drop('RETAIL_PRICE', inplace=True, axis=1)
train.drop('PROMO_PRICE', inplace=True, axis=1)
train.drop('COMPETITOR_PRICE', inplace=True, axis=1)
train.drop('Inventory', inplace=True, axis=1)
train.drop('GSPI', inplace=True, axis=1)
train.drop('INDEX OF CONSUMER SENTIMENT', inplace=True, axis=1)
train.drop('SALES_YEAR', inplace=True, axis=1)
train.drop('SALES_MONTH', inplace=True, axis=1)
train.drop('SALES_DAY', inplace=True, axis=1)
train.drop('SALES_WEEKDAY', inplace=True, axis=1)

In [8]:
#add the imputed values to the training set
train_new = pd.concat([train, missing_values_1], axis = 0)
print(train_new.shape)
train_new.head()

(879429, 3)


,Encoded_SKU_ID,SALES_DATE,DAILY_UNITS
0,224,2017-05-28,0
1,24,2017-05-28,0
2,463,2017-05-28,0
3,9,2017-05-28,0
4,2,2017-05-28,0


In [9]:
#reset the index to the time series sales date column
df = train_new.loc[train_new['Encoded_SKU_ID'] == 2]
print(df.shape)
df.drop(columns = 'Encoded_SKU_ID', inplace= True)
df.set_index(df['SALES_DATE'], inplace=True, drop=True)
df.drop(columns = 'SALES_DATE', inplace= True)
df

(1891, 3)


,DAILY_UNITS
SALES_DATE,
2017-05-28,0
2017-05-29,0
2017-05-30,0
2017-05-31,0
2017-06-01,0
...,...
2022-07-27,5
2022-07-28,5
2022-07-29,3


In [10]:
#Create the Croston model function

#Croston's Method - https://medium.com/analytics-vidhya/croston-forecast-model-for-intermittent-demand-360287a17f5f
def Croston_TSB(ts,extra_periods=1,alpha=0.85,beta=0.5):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
    print(a.shape)
    print(p.shape)
    print(f.shape) 
    print(d.shape)                 
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [11]:
#create a Croston prediction for each SKU in the training dataframe
croston_series_all = []
sku_list = list(train_new['Encoded_SKU_ID'].unique())
for sku in sku_list:
    df = train_new.loc[train_new['Encoded_SKU_ID'] == sku]
    df.drop(columns= 'Encoded_SKU_ID', inplace= True)
    df.set_index(df['SALES_DATE'], inplace=True, drop=True)
    df.drop(columns= 'SALES_DATE', inplace= True)
    res= Croston_TSB(df,extra_periods=7)
    croston_series_all.append(res)

(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1897,)
(1897,)
(1897,)
(1897,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)


In [12]:
#add the SKU ID back to the Croston series and turn the Croston series into a dataframe
for i, df_i in enumerate(croston_series_all):
    df_i['Encoded_SKU_ID'] = sku_list[i]

croston_df = pd.DataFrame()
for x in croston_series_all:
    croston_df = pd.concat([croston_df, x],axis = 0)
croston_df.head()
print(croston_df.shape)

(883454, 6)


In [ ]:
#round the forecasts to nearest whole number, since you can't sell a fraction of an SKU
decimals = pd.Series([0], index=['Forecast'])
croston_df_updated = croston_df.round(decimals)

In [ ]:
#create a dataframe of just the next 7 day prediction to test accuracy of the model
croston_df_predicted = croston_df_updated[croston_df_updated['Demand'].isnull()]

In [24]:
#create a csv file out of the predictions
croston_df_predicted.to_csv('nextweek_predictions.csv')

In [13]:
#read the test data
test = pd.read_csv(
    open(
        r'C:\Users\cmgas\OneDrive\git\Best-Buy-Case\Validation_Data_csv.csv', 'rb'),
)

In [14]:
#clean and arrange the test data in the same way as the train data
test['SALES_DATE'] = pd.to_datetime(test['SALES_DATE'])

In [15]:
#replace NaN with 0s
test.replace(np.nan, 0, inplace= True)

In [16]:
#drop exogenous columns
test.drop(columns = ['SUBCLASS_NAME'], inplace=True, axis=1)
test.drop('CLASS_NAME', inplace=True, axis=1)
test.drop('ML_NAME', inplace=True, axis=1)
test.drop('CATEGORY_NAME', inplace=True, axis=1)
test.drop('RETAIL_PRICE', inplace=True, axis=1)
test.drop('PROMO_PRICE', inplace=True, axis=1)
test.drop('COMPETITOR_PRICE', inplace=True, axis=1)
test.drop('Inventory', inplace=True, axis=1)
test.drop('Forecasted Units', inplace=True, axis=1)

In [19]:
#add the missing values to the test dataset
missing_dates = []
missing_skus = []

for i in list(train_new['Encoded_SKU_ID'].unique()):
    df_filt = test[test['Encoded_SKU_ID'] == i]
    dates_present = df_filt['SALES_DATE'].unique()
    for date_val in list(test['SALES_DATE'].unique()):
        if date_val in dates_present:
            continue
        else:
            missing_dates.append(date_val)
            missing_skus.append(i)

In [20]:
missing_values = pd.DataFrame(zip(missing_dates, missing_skus), columns = ['SALES_DATE', 'Encoded_SKU_ID'])
missing_values['DAILY_UNITS'] = 0

In [21]:
test_new = pd.concat([test, missing_values], axis = 0)

In [22]:
#sort both the test and prediction dataframes by SKU ID to ensure they match to predict error
test_final = test_new.sort_values(['Encoded_SKU_ID', 'SALES_DATE'])
predicted_final = croston_df_predicted.sort_values('Encoded_SKU_ID')

In [23]:
from sklearn import metrics

mse = metrics.mean_squared_error(test_final['DAILY_UNITS'], predicted_final['Forecast'])
rmse = np.sqrt(mse)
print(rmse)

4.981281732833764
